In [11]:
import os
import pandas as pd
from pathlib import Path

# === CONFIG ===
folder_path = "data/inwent_tables_csv"
output_dir = "terraform/modules/generated"
Path(output_dir).mkdir(parents=True, exist_ok=True)

# === TYPE INFERENCE ===
def infer_dynamodb_type(series: pd.Series) -> str:
    if pd.api.types.is_integer_dtype(series) or pd.api.types.is_float_dtype(series):
        return "N"
    return "S"

def guess_hash_key(columns):
    candidates = [c for c in columns if c.lower().endswith("_id") or c.lower() == "id"]
    return candidates[0] if candidates else columns[0]

def sanitize_table_name(name: str) -> str:
    return name.lower().replace(" ", "_").replace("-", "_")

# === PROCESS EACH CSV FILE ===
for csv_file in Path(folder_path).glob("*.csv"):
    table_name = sanitize_table_name(csv_file.stem)
    df = pd.read_csv(csv_file, nrows=1000)

    attributes = [(col, infer_dynamodb_type(df[col])) for col in df.columns]
    hash_key = guess_hash_key(df.columns)

    attr_block = ",\n    ".join(
        [f'{{ name = "{col}", type = "{dtype}" }}' for col, dtype in attributes]
    )

    terraform_module = f"""
module "{table_name}_table" {{
  source     = "./modules/dynamodb_table"
  table_name = "{table_name}"
  hash_key   = "{hash_key}"

  attributes = [
    {attr_block}
  ]
}}
""".strip()

    # Save to file
    output_file = Path(output_dir) / f"{table_name}_table.tf"
    output_file.write_text(terraform_module)
    print(f"✅ Generated: {output_file}")


✅ Generated: terraform/modules/generated/towary_table.tf
✅ Generated: terraform/modules/generated/ceny_towarow_table.tf
✅ Generated: terraform/modules/generated/akt_stan_mag_table.tf
✅ Generated: terraform/modules/generated/klasy_towarow_table.tf
